## Web Scraping Weather data and visualization
- In this project we will scrape weather data from the website  **"http://www.estesparkweather.net/archive_reports.php?date=200901"**
- Scrape all the available attributes of weather data for each day from **2010-01-01 to 2023-01-31**
- Ignore records for missing days
- Represent the scraped data as **pandas dataframe** object.

### Dataframe specific deatails
- Column names:   
       ['Average temperature (°F)', 'Average humidity (%)',
       'Average dewpoint (°F)', 'Average barometer (in)',
       'Average windspeed (mph)', 'Average gustspeed (mph)',
       'Average direction (°deg)', 'Rainfall for month (in)',
       'Rainfall for year (in)', 'Maximum rain per minute',
       'Maximum temperature (°F)', 'Minimum temperature (°F)',
       'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
       'Minimum pressure', 'Maximum windspeed (mph)',
       'Maximum gust speed (mph)', 'Maximum heat index (°F)']
- Each record in the dataframe corresponds to weather deatils of a given day
- Index column is **date-time format (yyyy-mm-dd)**

In [0]:
#Header File
import bs4
from bs4 import BeautifulSoup
import csv
import requests
import time
import pandas as pd
import urllib
import re
import pickle
import mechanicalsoup
import re
import numpy as np
import datetime
from datetime import datetime
from datetime import date

In [0]:
# 1 - Soup  Initialization 
browser = mechanicalsoup.Browser(soup_config={'features': 'html.parser'})
url = "https://www.estesparkweather.net/archive_reports.php"
weather_date_page = browser.get(url)
page_in_html = weather_date_page.soup

# Function to refresh the page and fetch new HTML code for different date
def refresh_page(year_month):
    form = page_in_html.select("form")[0]
    form.select("option")[0]["value"] = year_month
    next_page = browser.submit(form, weather_date_page.url).soup
    return next_page

In [0]:
# DataFram to store the entire scrapped data
df = pd.DataFrame()

# DataFrame with Columns added
df = df.reindex(columns = ['Day','Average temperature', 'Average humidity',
 'Average dewpoint', 'Average barometer',
 'Average windspeed', 'Average gustspeed',
 'Average direction', 'Rainfall for month',
 'Rainfall for year', 'Maximum rain per minute',
 'Maximum temperature', 'Minimum temperature',
 'Maximum humidity', 'Minimum humidity', 'Maximum pressure',
 'Minimum pressure', 'Maximum windspeed',
 'Maximum gust speed', 'Maximum heat index'])

df.head()

,Day,Average temperature,Average humidity,Average dewpoint,Average barometer,Average windspeed,Average gustspeed,Average direction,Rainfall for month,Rainfall for year,Maximum rain per minute,Maximum temperature,Minimum temperature,Maximum humidity,Minimum humidity,Maximum pressure,Minimum pressure,Maximum windspeed,Maximum gust speed,Maximum heat index


In [0]:
# Dates to loop
dates= pd.date_range('2010-01-01', '2023-01-31', freq='M').strftime('%Y%m').tolist()

# Fetching data and appending to DataFram df
for dt in dates:
    html_code = refresh_page(dt)
    pattern = re.compile(r'.*\b(Average and Extremes)$')
    for i in range(0,len(html_code.find_all('table'))):
        td_value = 'empty' if html_code.find_all('table')[i].td.string is None else html_code.find_all('table')[i].td.string
        if(pattern.match(td_value)):
            len_tr = len(html_code.find_all('table')[i].find_all('tr'))
            datestr = str(html_code.find_all('table')[i].find_all('td' ,attrs={'colspan':2})[0].string)
            md = str(re.findall('^\w* \d+',datestr)[0])
            dat = ' '.join([md,dt[0:-2]])
            date_obj = datetime.strptime(dat,"%b %d %Y")
            fmt_date = date_obj.strftime('%Y-%m-%d')            
            dist = {}
            dist.update({'Day':fmt_date})
            for m in range(0,len_tr):
                len_td = len(html_code.find_all('table')[i].find_all('tr')[m].find_all('td'))
                for n in range(0,len_td):
                    if n!=0:
                        temp_str= str(html_code.find_all('table')[i].find_all('tr')[m].find_all('td')[n].string).strip()
                        corr_val = float(re.findall('^-?\d*\.?\d+',temp_str)[0])
                        string=html_code.find_all('table')[i].find_all('tr')[m].find_all('td')[0].string
#                         print('{} : {}'.format(html_code.find_all('table')[27].find_all('tr')[m].find_all('td')[0].string,corr_val))
                        dist.update({string:corr_val})

            df = df.append(dist,ignore_index=True)
df

Exception ignored in: <function Browser.__del__ at 0x7f836bdcf700>
Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/mechanicalsoup/browser.py", line 342, in __del__
    self._finalize()
AttributeError: 'Browser' object has no attribute '_finalize'


,Day,Average temperature,Average humidity,Average dewpoint,Average barometer,Average windspeed,Average gustspeed,Average direction,Rainfall for month,Rainfall for year,Maximum rain per minute,Maximum temperature,Minimum temperature,Maximum humidity,Minimum humidity,Maximum pressure,Minimum pressure,Maximum windspeed,Maximum gust speed,Maximum heat index
0,2010-01-01,32.1,49.0,15.2,30.2,14.6,19.7,297.0,0.00,0.00,0.00,40.0,22.1,61.0,41.0,30.330,30.034,34.5,51.0,40.0
1,2010-01-02,32.1,50.0,15.5,30.1,8.8,12.2,306.0,0.00,0.00,0.00,39.1,22.0,77.0,39.0,30.292,29.937,23.0,35.0,39.1
2,2010-01-03,23.1,64.0,12.1,30.4,2.9,4.4,21.0,0.00,0.00,0.00,33.4,9.2,85.0,34.0,30.549,30.237,17.3,24.0,33.4
3,2010-01-04,25.7,48.0,7.2,30.4,4.7,7.0,324.0,0.00,0.00,0.00,36.5,7.6,82.0,25.0,30.585,30.107,15.0,21.0,36.5
4,2010-01-05,34.3,51.0,17.8,30.0,14.2,18.5,300.0,0.00,0.00,0.00,40.1,28.4,60.0,43.0,30.173,29.897,34.5,43.0,40.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446,2023-01-27,26.8,55.0,12.6,29.8,13.6,19.9,254.0,0.24,0.52,0.00,33.2,23.7,81.0,43.0,30.063,29.687,26.5,40.3,33.2
4447,2023-01-28,21.0,69.0,11.9,29.8,6.8,10.5,286.0,0.24,0.52,0.00,32.5,3.1,89.0,50.0,29.963,29.604,23.0,33.4,32.5
4448,2023-01-29,-0.8,80.0,-5.4,30.1,3.6,5.5,52.0,0.24,0.52,0.00,7.2,-9.5,84.0,77.0,30.396,29.845,10.4,15.0,7.2
4449,2023-01-30,-0.1,82.0,-4.3,30.4,1.3,2.2,48.0,0.24,0.52,0.00,9.5,-9.4,85.0,79.0,30.461,30.335,5.8,8.1,9.5


In [0]:
# Creating new dict for renaming columns
rename_col={}

# New column names
new_col =['Average temperature (°F)', 'Average humidity (%)',
 'Average dewpoint (°F)', 'Average barometer (in)',
 'Average windspeed (mph)', 'Average gustspeed (mph)',
 'Average direction (°deg)', 'Rainfall for month (in)',
 'Rainfall for year (in)', 'Maximum rain per minute',
 'Maximum temperature (°F)', 'Minimum temperature (°F)',
 'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
 'Minimum pressure', 'Maximum windspeed (mph)',
 'Maximum gust speed (mph)', 'Maximum heat index (°F)']

# Old column names
old_col = ['Average temperature', 'Average humidity',
 'Average dewpoint', 'Average barometer',
 'Average windspeed', 'Average gustspeed',
 'Average direction', 'Rainfall for month',
 'Rainfall for year', 'Maximum rain per minute',
 'Maximum temperature', 'Minimum temperature',
 'Maximum humidity', 'Minimum humidity', 'Maximum pressure',
 'Minimum pressure', 'Maximum windspeed',
 'Maximum gust speed', 'Maximum heat index']

# zipping them to new dict
rename_col.update(zip(old_col, new_col))

#Renaming Columns 
df.rename(columns = rename_col,inplace=True)

df

,Day,Average temperature (°F),Average humidity (%),Average dewpoint (°F),Average barometer (in),Average windspeed (mph),Average gustspeed (mph),Average direction (°deg),Rainfall for month (in),Rainfall for year (in),Maximum rain per minute,Maximum temperature (°F),Minimum temperature (°F),Maximum humidity (%),Minimum humidity (%),Maximum pressure,Minimum pressure,Maximum windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F)
0,2010-01-01,32.1,49.0,15.2,30.2,14.6,19.7,297.0,0.00,0.00,0.00,40.0,22.1,61.0,41.0,30.330,30.034,34.5,51.0,40.0
1,2010-01-02,32.1,50.0,15.5,30.1,8.8,12.2,306.0,0.00,0.00,0.00,39.1,22.0,77.0,39.0,30.292,29.937,23.0,35.0,39.1
2,2010-01-03,23.1,64.0,12.1,30.4,2.9,4.4,21.0,0.00,0.00,0.00,33.4,9.2,85.0,34.0,30.549,30.237,17.3,24.0,33.4
3,2010-01-04,25.7,48.0,7.2,30.4,4.7,7.0,324.0,0.00,0.00,0.00,36.5,7.6,82.0,25.0,30.585,30.107,15.0,21.0,36.5
4,2010-01-05,34.3,51.0,17.8,30.0,14.2,18.5,300.0,0.00,0.00,0.00,40.1,28.4,60.0,43.0,30.173,29.897,34.5,43.0,40.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446,2023-01-27,26.8,55.0,12.6,29.8,13.6,19.9,254.0,0.24,0.52,0.00,33.2,23.7,81.0,43.0,30.063,29.687,26.5,40.3,33.2
4447,2023-01-28,21.0,69.0,11.9,29.8,6.8,10.5,286.0,0.24,0.52,0.00,32.5,3.1,89.0,50.0,29.963,29.604,23.0,33.4,32.5
4448,2023-01-29,-0.8,80.0,-5.4,30.1,3.6,5.5,52.0,0.24,0.52,0.00,7.2,-9.5,84.0,77.0,30.396,29.845,10.4,15.0,7.2
4449,2023-01-30,-0.1,82.0,-4.3,30.4,1.3,2.2,48.0,0.24,0.52,0.00,9.5,-9.4,85.0,79.0,30.461,30.335,5.8,8.1,9.5


In [0]:
ndf = df[0:-3].copy()
print(ndf.isnull().sum())

Day                         0
Average temperature (°F)    0
Average humidity (%)        0
Average dewpoint (°F)       0
Average barometer (in)      0
Average windspeed (mph)     0
Average gustspeed (mph)     0
Average direction (°deg)    0
Rainfall for month (in)     0
Rainfall for year (in)      0
Maximum rain per minute     0
Maximum temperature (°F)    0
Minimum temperature (°F)    0
Maximum humidity (%)        0
Minimum humidity (%)        0
Maximum pressure            0
Minimum pressure            0
Maximum windspeed (mph)     0
Maximum gust speed (mph)    0
Maximum heat index (°F)     0
dtype: int64


In [0]:
# setting index 
ndf['Day'] = pd.to_datetime(ndf['Day'],format='%Y-%m-%d')
idx_df = ndf.set_index('Day')

#changing the type as datetime
idx_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4448 entries, 2010-01-01 to 2023-01-28
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Average temperature (°F)  4448 non-null   float64
 1   Average humidity (%)      4448 non-null   float64
 2   Average dewpoint (°F)     4448 non-null   float64
 3   Average barometer (in)    4448 non-null   float64
 4   Average windspeed (mph)   4448 non-null   float64
 5   Average gustspeed (mph)   4448 non-null   float64
 6   Average direction (°deg)  4448 non-null   float64
 7   Rainfall for month (in)   4448 non-null   float64
 8   Rainfall for year (in)    4448 non-null   float64
 9   Maximum rain per minute   4448 non-null   float64
 10  Maximum temperature (°F)  4448 non-null   float64
 11  Minimum temperature (°F)  4448 non-null   float64
 12  Maximum humidity (%)      4448 non-null   float64
 13  Minimum humidity (%)      4448 non-null   flo

In [0]:
import pickle
with open("dataframe.pk", "wb") as file:
    pickle.dump(idx_df, file)

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkSession').getOrCreate()

In [0]:
py_df = spark.createDataFrame(ndf)
py_df.printSchema()


root
 |-- Day: timestamp (nullable = true)
 |-- Average temperature (°F): double (nullable = true)
 |-- Average humidity (%): double (nullable = true)
 |-- Average dewpoint (°F): double (nullable = true)
 |-- Average barometer (in): double (nullable = true)
 |-- Average windspeed (mph): double (nullable = true)
 |-- Average gustspeed (mph): double (nullable = true)
 |-- Average direction (°deg): double (nullable = true)
 |-- Rainfall for month (in): double (nullable = true)
 |-- Rainfall for year (in): double (nullable = true)
 |-- Maximum rain per minute: double (nullable = true)
 |-- Maximum temperature (°F): double (nullable = true)
 |-- Minimum temperature (°F): double (nullable = true)
 |-- Maximum humidity (%): double (nullable = true)
 |-- Minimum humidity (%): double (nullable = true)
 |-- Maximum pressure: double (nullable = true)
 |-- Minimum pressure: double (nullable = true)
 |-- Maximum windspeed (mph): double (nullable = true)
 |-- Maximum gust speed (mph): double (nullab